# Scenario 1: Forecasting Food Safety Incidents
This notebook is complementary material for the walkthrough scenario "Forecasting Food Safety Incidents" used the STELAR KLMS
It is not intended to be run as a standalone notebook. It **requires access to a deployment of STELAR KLMS** and an **account** on the respective instance. 

Some of the instances used during the evaluation period of the STELAR Project are:

Internal Pilot Instance: https://klms.stelar.gr

Public Sandbox Instance: https://sandbox.stelar.gr


*If you don't have an account on the STELAR KLMS, you can create one on the respective instance. 
Kindly note that the internal pilot instance is only accessible to STELAR project members, while the public sandbox instance is open to everyone by registration.*

---
# Overview

This notebook is intended to run the Forecasting Models Orchestrator (FOMO) in order to produce forecasts for food safety incidents.

### Prerequisites

- Fill in your accounts credentials in the block below. 
- Select datasets according to the walkthrough directions.
- Ensure you have a modern python version installed (3.9 or later).
- Install the STELAR Python SDK and any other required libraries (`pip install stelar_client --upgrade`).

### Instatiate a STELAR Client object
**Modify credentials and base URL as needed.**

In [ ]:
from stelar.client import Client, Dataset, TaskSpec, Process
from datetime import datetime

# Base URL
# Sandbox: https://sandbox.stelar.gr
# Internal Pilots: https://klms.stelar.gr

BASE_URL = "https://sandbox.stelar.gr"
USERNAME = "your_username"  # Replace with your username
PASSWORD = "your_password"  # Replace with your password

c = Client(base_url=BASE_URL, username=USERNAME, password=PASSWORD)
print(f"Connected to STELAR KLMS @ {c._base_url} as {c._username}")

### Select a food safety incidents dataset 

In [ ]:
food_safety_dataset = c.datasets["food-safety-incidents-products-and-hazards"]
print(f"Selected Dataset: {food_safety_dataset.id} | {food_safety_dataset.title}")
print(f"Browse the dataset at: {c._base_url}/console/v1/catalog/{food_safety_dataset.id}")

### Create/Select a Workflow Process to run the forecasting task

In [ ]:
ORGANIZATION = "stelar-klms"

try:
    c.processes.create(**{
        "title": "Evalution Workflow for " + c._username,
        "name": "evaluation-workflow-" + c._username,
        "organization": c.organizations[ORGANIZATION]
    })
    proc = c.processes["evaluation-workflow-" + c._username]
    print(f"Created new process for evaluation: {c.processes[-1].id} | {c.processes[-1].title}")
except Exception as e:
    proc = c.processes["evaluation-workflow-" + c._username]
    print(f"Using existing process for evaluation: {proc.id} | {proc.title}")

### Create a dataset to store the results of the forecasting task

In [ ]:
ORGANIZATION = "stelar-klms"

try:
    res_dset = c.datasets.create(**{
        "title": "Food Incidents Forecast for " + c._username,
        "name": "food-incidents-forecast-" + c._username,
        "organization": c.organizations[ORGANIZATION],
        "notes": "Forecasting food safety incidents based on historical data curated by " + c._username,
    })
    print(f"Created new dataset for food incidents forecast: {c.datasets[-1].id} | {c.datasets[-1].title}")
except Exception as e:
    res_dset = c.datasets["food-incidents-forecast-" + c._username]
    print(f"Using existing dataset for food incidents forecast: {res_dset.id} | {res_dset.title}")

### Prepare & Run the forecasting task

In [ ]:
t = TaskSpec(tool="fomo", name="Food Incidents Forecasting for "+c._username)
proc = c.processes["evaluation-workflow-" + c._username]

# Create local dataset aliases for the task 
t.d(alias="d0", dset=res_dset.id)

# Define the input, in this case the food safety dataset

# Choose
#   - food_safety_dataset.resources[0] if you want to use the resource for weekly incidents 
#   - food_safety_dataset.resources[1] if you want to use the resource for monthly incidents
#
t.i(stream=food_safety_dataset.resources[1])
# Define the output path, where the predictions will be stored

# Create a timestamp for the output files so they are distinguishable and not overwrite each other

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

t.o(predictions={
        "url": "s3://klms-bucket/evaluation/experiments/proc-" + str(proc.id) + "/predictions_"+c._username+"_"+timestamp+".csv",
        "dataset": "d0",
        "resource": {
            "name": "Food Safety Incidents Predictions for " + c._username,
            "format": "csv",
            "relation": "predictions",
        }
    },
    log={
        "url": "s3://klms-bucket/evaluation/experiments/proc-" + str(proc.id) + "/log_"+c._username+"_"+timestamp+".txt",
        "dataset": "d0",
        "resource": {
            "name": "Food Safety Incidents Predictions Log for " + c._username,
            "format": "txt",
            "relation": "log",
        }
    })

# Run the task using the workflow process created earlier
predictions_task = proc.run(t)
print(f"Task {predictions_task.id} is running. Check the status at: {c._base_url}/console/v1/task/{str(proc.id)}/{str(predictions_task.id)}")